In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import sort
#import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import random, math
from sklearn import metrics, preprocessing

In [2]:
data_path = "F:\\Users\\ivann_000\\AnacondaProjects\\MLBootcamp4\\data"
X = pd.read_csv(data_path+"\\x_train.csv", sep=';', header=None)
y = pd.read_csv(data_path+"\\y_train.csv", sep=';', header=None)[0]
#X = preprocessing.scale(X)
#X = preprocessing.normalize(X)
XX = pd.DataFrame(X)
XX[223] = y
len(X), len(y), XX

(3489,
 3489,
            0         1         2         3         4          5         6    \
 0    -1.351735  1.502242  2.049512  0.437597  0.243818   4.982503 -1.493039   
 1     2.810473  1.312591  1.392652  0.163840  0.654384  -2.725782  3.195347   
 2     2.328788 -1.928459 -2.064532  0.731323  0.117712  -1.567757  2.961654   
 3    -0.128106 -2.072688 -2.407602  0.978550  0.111442   2.608775 -1.299084   
 4     1.886821  0.757923 -0.097547  0.465719  0.621116  -0.332968  2.446465   
 5    -1.093613  1.747583  2.499609  0.366799  0.888956   3.371590 -1.416785   
 6     0.717601 -1.307117 -2.156820  0.337006  0.071711   1.920241  0.730820   
 7     4.600603 -1.605449 -1.889961  0.945001  0.637761  -0.853413  5.844666   
 8    -0.842231  2.782335  3.072997  0.314701  0.344247   1.278883 -1.316395   
 9    -0.712364  0.531405  0.466771  0.123207  0.588291   1.935748 -1.307892   
 10   -0.353339  1.124631  1.701043  0.890847  0.165852   0.848457 -0.099305   
 11    3.043221 -1.368781 

In [4]:
labels = [0, 1, 2, 3, 4]
c_count = [len(y[y==i]) for i in labels]
c_count

[113, 1164, 1521, 581, 110]

In [5]:
XX_count = len(XX[0])
max_samples = max(c_count)
for label in labels:
    cl0 = XX[XX[223]==label].drop([223], axis=1)
    stds = [cl0[e].std()/20 for e in cl0]
    row_count = len(cl0[0])
    values = cl0.values
    while row_count < max_samples:
        print(row_count)
        row = []
        cl0_row = random.choice(values)
        for i, col in enumerate(cl0_row):
            row.append(random.normalvariate(col, stds[i]))
        row.append(label)
        XX.loc[XX_count] = row
        XX_count += 1
        row_count += 1
len(XX[0])

113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362


7605

In [6]:
X_new = XX.values[:,0:222]
y_new = XX.values[:,223]
X = pd.read_csv(data_path+"\\x_train.csv", sep=';', header=None).values[:,0:222]
y = pd.read_csv(data_path+"\\y_train.csv", sep=';', header=None).values[:,0]

In [7]:
len(X_new), len(X), len(y_new), len(y)

(7605, 3489, 7605, 3489)

In [13]:
len(random.choice(XX[XX[223]==0].drop([223], axis=1).values))

223

In [11]:
model = XGBClassifier(learning_rate =0.01,
                      n_estimators=200,
                      max_depth=7,
                      min_child_weight=1,
                      gamma=0,
                      subsample=0.75,
                      colsample_bytree=0.75,
                      seed=27)
model.fit(X_new, y_new)
print(model)
expected = y
predicted = model.predict(X)
print(metrics.classification_report(expected, predicted))
# cross_val_score(model, X_new, y_new, scoring='accuracy', cv=3).mean()

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.75)
             precision    recall  f1-score   support

          0       0.96      0.99      0.97       113
          1       0.86      0.89      0.87      1164
          2       0.88      0.88      0.88      1521
          3       0.96      0.88      0.92       581
          4       0.99      0.95      0.97       110

avg / total       0.89      0.89      0.89      3489



In [12]:
metrics.accuracy_score(expected, predicted)
# metrics.confusion_matrix(expected, predicted)

0.88907996560619085

In [10]:
X_test = pd.read_csv(data_path+"\\x_test.csv", sep=';', header=None).values[:,0:222]
predicted = model.predict(X_test)
df = pd.DataFrame(predicted)
df = df.astype(int)
df.to_csv(data_path+"\\y_test2.csv", header=None, index=None, encoding='utf-8')

XGBoostError: need to call fit beforehand